In [ ]:
# COMP 548 - MEDICAL IMAGE ANALYSIS HOMEWORK #3

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 14.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.autograd import Variable

import torchvision
from torchvision import models
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, ToPILImage
from torchvision.io import read_image
from torchvision import transforms
from torchmetrics.classification import BinaryPrecision, BinaryRecall, BinaryF1Score
import statistics

import torch.utils.data
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, Subset
from torch.utils.data import WeightedRandomSampler

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score


import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import copy
import multiprocessing as mp
from tqdm import tqdm

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

mp.set_start_method('spawn', force=True)

In [ ]:
im_path = '.../path/...'
im = Image.open(im_path)
image_array = np.array(im)
print(image_array.max())
print(image_array.min())
print(im.mode)
_, file_extension = os.path.splitext(im_path)
print(file_extension)

253
0
L
.jpg


In [ ]:
im_gold_path = '.../path/...'
im_gold = Image.open(im_gold_path)
image_gold_array = np.array(im_gold)
print(image_gold_array.max())
print(image_gold_array.min())
print(im_gold.mode)
_, file_gold_extension = os.path.splitext(im_gold_path)
print(file_gold_extension)

1
0
L
.png


In [ ]:
unique_values = np.unique(image_gold_array)
print(f"Gold standard unique values: {unique_values}")

Gold standard unique values: [0 1]


In [ ]:
training_dataset_dir = '.../path/...'
validation_dataset_dir = '.../path/...'
test_dataset_dir = '.../path/...'

gold_training_dir = '.../path/...'
gold_validation_dir = '.../path/...'
gold_test_dir = '.../path/...'

In [ ]:
class MyDataset(Dataset):
    def __init__(self, image_dir, gold_dir):
        self.image_dir = image_dir
        self.gold_dir = gold_dir
        self.images = sorted(os.listdir(image_dir))
        self.gold_images = sorted(os.listdir(gold_dir))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.images[idx])
        gold_name = os.path.join(self.gold_dir, self.gold_images[idx])

        image = Image.open(img_name)
        gold_standard = Image.open(gold_name)

        image_np = np.array(image).astype(np.float32) / 255.0  # Normalize to [0, 1]
        gold_standard_np = np.array(gold_standard).astype(np.float32)  # Do not normalize gold standards!!!!!!

        image_tensor = torch.from_numpy(image_np).unsqueeze(0)
        gold_standard_tensor = torch.from_numpy(gold_standard_np).unsqueeze(0)

        return image_tensor, gold_standard_tensor, self.images[idx], self.gold_images[idx]

In [ ]:
train_dataset = MyDataset(training_dataset_dir, gold_training_dir)
val_dataset = MyDataset(validation_dataset_dir, gold_validation_dir)
test_dataset = MyDataset(test_dataset_dir, gold_test_dir)

In [ ]:
ct_image_filenames = test_dataset.images
print("CT Image Filenames:")
print(ct_image_filenames)

gold_image_filenames = test_dataset.gold_images
print("Gold Standard Image Filenames:")
print(gold_image_filenames)

CT Image Filenames:
['CT_11_Image_47.jpg', 'CT_11_Image_48.jpg', 'CT_11_Image_49.jpg', 'CT_11_Image_50.jpg', 'CT_11_Image_51.jpg', 'CT_11_Image_52.jpg', 'CT_11_Image_53.jpg', 'CT_11_Image_54.jpg', 'CT_11_Image_55.jpg', 'CT_11_Image_56.jpg', 'CT_11_Image_57.jpg', 'CT_11_Image_58.jpg', 'CT_11_Image_59.jpg', 'CT_11_Image_60.jpg', 'CT_11_Image_61.jpg', 'CT_11_Image_62.jpg', 'CT_12_Image_49.jpg', 'CT_12_Image_50.jpg', 'CT_12_Image_51.jpg', 'CT_12_Image_52.jpg', 'CT_12_Image_53.jpg', 'CT_12_Image_54.jpg', 'CT_12_Image_55.jpg', 'CT_12_Image_56.jpg', 'CT_12_Image_57.jpg', 'CT_12_Image_58.jpg', 'CT_12_Image_59.jpg', 'CT_12_Image_60.jpg', 'CT_12_Image_61.jpg', 'CT_13_Image_42.jpg', 'CT_13_Image_43.jpg', 'CT_13_Image_44.jpg', 'CT_13_Image_45.jpg', 'CT_13_Image_46.jpg', 'CT_13_Image_47.jpg', 'CT_13_Image_48.jpg', 'CT_13_Image_49.jpg', 'CT_13_Image_50.jpg', 'CT_13_Image_51.jpg', 'CT_13_Image_52.jpg', 'CT_13_Image_53.jpg', 'CT_13_Image_54.jpg', 'CT_13_Image_55.jpg', 'CT_13_Image_56.jpg', 'CT_13_Imag

In [ ]:
image, gold_standard, _, _ = train_dataset[0]
print(f"Image shape: {image.shape}, dtype: {image.dtype}")
print(f"Gold standard shape: {gold_standard.shape}, dtype: {gold_standard.dtype}")

Image shape: torch.Size([1, 512, 512]), dtype: torch.float32
Gold standard shape: torch.Size([1, 512, 512]), dtype: torch.float32


In [ ]:
image, gold_standard, _, _ = train_dataset[55]

image_unique_values = torch.unique(image)
gold_unique_values = torch.unique(gold_standard)

print(f"Unique values in CT image: {image_unique_values}")
print(f"Unique values in gold_standard: {gold_unique_values}")

Unique values in CT image: tensor([0.0000, 0.0039, 0.0078, 0.0118, 0.0157, 0.0196, 0.0235, 0.0275, 0.0314,
        0.0353, 0.0392, 0.0431, 0.0471, 0.0510, 0.0549, 0.0588, 0.0627, 0.0667,
        0.0706, 0.0745, 0.0784, 0.0824, 0.0863, 0.0902, 0.0941, 0.0980, 0.1020,
        0.1059, 0.1098, 0.1137, 0.1176, 0.1216, 0.1255, 0.1294, 0.1333, 0.1373,
        0.1412, 0.1451, 0.1490, 0.1529, 0.1569, 0.1608, 0.1647, 0.1686, 0.1725,
        0.1765, 0.1804, 0.1843, 0.1882, 0.1922, 0.1961, 0.2000, 0.2039, 0.2078,
        0.2118, 0.2157, 0.2196, 0.2235, 0.2275, 0.2314, 0.2353, 0.2392, 0.2431,
        0.2471, 0.2510, 0.2549, 0.2588, 0.2627, 0.2667, 0.2706, 0.2745, 0.2784,
        0.2824, 0.2863, 0.2902, 0.2941, 0.2980, 0.3020, 0.3059, 0.3098, 0.3137,
        0.3176, 0.3216, 0.3255, 0.3294, 0.3333, 0.3373, 0.3412, 0.3451, 0.3490,
        0.3529, 0.3569, 0.3608, 0.3647, 0.3686, 0.3725, 0.3765, 0.3804, 0.3843,
        0.3882, 0.3922, 0.3961, 0.4000, 0.4039, 0.4078, 0.4118, 0.4157, 0.4196,
        0.423

In [ ]:
def imshow_mydata(image, gold_standard):
    image_np = image.squeeze().numpy()
    gold_standard_np = gold_standard.squeeze().numpy()

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    axes[0].imshow(image_np, cmap='gray')
    axes[0].set_title(f'Normalized CT Image')
    axes[0].axis('off')

    axes[1].imshow(gold_standard_np, cmap='gray')
    axes[1].set_title(f'Gold Standard')
    axes[1].axis('off')

    plt.show()

image, gold_standard, _, _ = train_dataset[1]
imshow_mydata(image, gold_standard)

In [ ]:
batch_size = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
# https://github.com/milesial/Pytorch-UNet/blob/master/unet/unet_parts.py
# https://github.com/Miltos-90/UNet_Biomedical_Image_Segmentation/blob/main/Unet_main.ipynb

In [ ]:
# ORIGINAL ARCH IN THE UNET PAPER

import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3),
                                         nn.ReLU(inplace=True),
                                         nn.Conv2d(out_channels, out_channels, kernel_size=3),
                                         nn.ReLU(inplace=True))

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(nn.MaxPool2d(2),
                                          DoubleConv(in_channels, out_channels))

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x2 = x2[:, :, diffY // 2:x2.size()[2] - diffY // 2, diffX // 2:x2.size()[3] - diffX // 2]

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        self.up1 = Up(1024, 512)
        self.up2 = Up(512, 256)
        self.up3 = Up(256, 128)
        self.up4 = Up(128, 64)
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        print("x1: ", x1.shape)
        x2 = self.down1(x1)
        print("x2: ", x2.shape)
        x3 = self.down2(x2)
        print("x3: ", x3.shape)
        x4 = self.down3(x3)
        print("x4: ", x4.shape)
        x5 = self.down4(x4)
        print("x5: ", x5.shape)
        x = self.up1(x5, x4)
        print("x: ", x.shape)
        x = self.up2(x, x3)
        print("x: ", x.shape)
        x = self.up3(x, x2)
        print("x: ", x.shape)
        x = self.up4(x, x1)
        print("x: ", x.shape)
        logits = self.outc(x)
        return logits

model = UNet(n_channels=1, n_classes=2)
input = torch.rand(1, 1, 572, 572)
output = model(input)
print("Output dimensions:", output.shape)

x1:  torch.Size([1, 64, 568, 568])
x2:  torch.Size([1, 128, 280, 280])
x3:  torch.Size([1, 256, 136, 136])
x4:  torch.Size([1, 512, 64, 64])
x5:  torch.Size([1, 1024, 28, 28])
x:  torch.Size([1, 512, 52, 52])
x:  torch.Size([1, 256, 100, 100])
x:  torch.Size([1, 128, 196, 196])
x:  torch.Size([1, 64, 388, 388])
Output dimensions: torch.Size([1, 2, 388, 388])


In [ ]:
# Default UNet with Batch Normalization

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                                         nn.BatchNorm2d(out_channels),
                                         nn.ReLU(inplace=True),
                                         nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                                         nn.BatchNorm2d(out_channels),
                                         nn.ReLU(inplace=True))

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(nn.MaxPool2d(2),
                                          DoubleConv(in_channels, out_channels))

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x2 = x2[:, :, diffY // 2:x2.size()[2] - diffY // 2, diffX // 2:x2.size()[3] - diffX // 2]
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class default_unet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(default_unet, self).__init__()
        self.inc = DoubleConv(n_channels, 16)
        self.down1 = Down(16, 32)
        self.down2 = Down(32, 64)
        self.down3 = Down(64, 128)
        self.down4 = Down(128, 256)
        self.up1 = Up(256, 128)
        self.up2 = Up(128, 64)
        self.up3 = Up(64, 32)
        self.up4 = Up(32, 16)
        self.outc = nn.Conv2d(16, n_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

model = default_unet(n_channels=1, n_classes=1)
input = torch.rand(1, 1, 512, 512)
output = model(input)
print("Output dimensions:", output.shape)

Output dimensions: torch.Size([1, 1, 512, 512])


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, num_epochs=30, patience=5, threshold = 0.5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_model_wts = None
    best_f1 = 0.0
    best_epoch = 0
    patience_counter = 0

    all_train_losses = []
    all_val_losses = []
    all_train_precisions = []
    all_val_precisions = []
    all_train_recalls = []
    all_val_recalls = []
    all_train_f1s = []
    all_val_f1s = []

    precision_metric = BinaryPrecision(threshold).to(device)
    recall_metric = BinaryRecall(threshold).to(device)
    f1_metric = BinaryF1Score(threshold).to(device)

    print("Training started:\n")

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        epoch_train_precisions, epoch_train_recalls, epoch_train_f1s = [], [], []

        for images, masks, _, _ in tqdm(train_loader):
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            train_outputs = torch.sigmoid(outputs)
            train_preds = train_outputs > threshold

            # Calculate and accumulate metrics
            precision = precision_metric(train_preds, masks)
            recall = recall_metric(train_preds, masks)
            f1_score = f1_metric(train_preds, masks)

            epoch_train_precisions.append(precision.item())
            epoch_train_recalls.append(recall.item())
            epoch_train_f1s.append(f1_score.item())

        # Compute average metrics for training
        avg_train_precision = sum(epoch_train_precisions) / len(epoch_train_precisions)
        avg_train_recall = sum(epoch_train_recalls) / len(epoch_train_recalls)
        avg_train_f1 = sum(epoch_train_f1s) / len(epoch_train_f1s)
        avg_train_loss = train_loss / len(train_loader)

        # Store the metrics for later averaging
        all_train_losses.append(avg_train_loss)
        all_train_precisions.append(avg_train_precision)
        all_train_recalls.append(avg_train_recall)
        all_train_f1s.append(avg_train_f1)

        print(f"Epoch {epoch+1}/{num_epochs} Train Loss: {avg_train_loss:.4f}")
        print(f"Train Pixel-level Precision: {avg_train_precision:.4f}")
        print(f"Train Recall: {avg_train_recall:.4f}")
        print(f"Train F score: {avg_train_f1:.4f}\n")

        precision_metric.reset()
        recall_metric.reset()
        f1_metric.reset()

        # Validation
        model.eval()
        val_loss = 0
        epoch_val_precisions, epoch_val_recalls, epoch_val_f1s = [], [], []

        with torch.no_grad():
            for val_images, val_masks, _, _ in val_loader:
                val_images = val_images.to(device)
                val_masks = val_masks.to(device)

                val_outputs = model(val_images)
                loss = criterion(val_outputs, val_masks)
                val_loss += loss.item()

                val_outputs = torch.sigmoid(val_outputs)
                val_preds = val_outputs > threshold

                # Calculate and accumulate metrics
                precision = precision_metric(val_preds, val_masks)
                recall = recall_metric(val_preds, val_masks)
                f1_score = f1_metric(val_preds, val_masks)

                epoch_val_precisions.append(precision.item())
                epoch_val_recalls.append(recall.item())
                epoch_val_f1s.append(f1_score.item())

        # Compute average metrics for validation
        avg_val_precision = sum(epoch_val_precisions) / len(epoch_val_precisions)
        avg_val_recall = sum(epoch_val_recalls) / len(epoch_val_recalls)
        avg_val_f1 = sum(epoch_val_f1s) / len(epoch_val_f1s)
        avg_val_loss = val_loss / len(val_loader)

        # Store the metrics for later averaging
        all_val_losses.append(avg_val_loss)
        all_val_precisions.append(avg_val_precision)
        all_val_recalls.append(avg_val_recall)
        all_val_f1s.append(avg_val_f1)

        print(f"Validation Loss: {avg_val_loss:.4f}")
        print(f"Validation Pixel-level Precision: {avg_val_precision:.4f}")
        print(f"Validation Recall: {avg_val_recall:.4f}")
        print(f"Validation F score: {avg_val_f1:.4f}\n")

        precision_metric.reset()
        recall_metric.reset()
        f1_metric.reset()

        # Where to save the model
        if avg_val_f1 > best_f1:
            best_f1 = avg_val_f1
            best_model_wts = model.state_dict()
            best_epoch = epoch + 1
            patience_counter = 0
        else:
            patience_counter += 1

        if scheduler is not None:
            scheduler.step(avg_val_loss)

        # Early stopping
        if patience_counter >= patience:
            print('Early stopping triggered')
            break

    if best_model_wts is not None:
        model.load_state_dict(best_model_wts)

    # Calculate average metrics over all epochs
    final_avg_train_loss = sum(all_train_losses) / len(all_train_losses)
    final_avg_train_precision = sum(all_train_precisions) / len(all_train_precisions)
    final_avg_train_recall = sum(all_train_recalls) / len(all_train_recalls)
    final_avg_train_f1 = sum(all_train_f1s) / len(all_train_f1s)

    final_avg_val_loss = sum(all_val_losses) / len(all_val_losses)
    final_avg_val_precision = sum(all_val_precisions) / len(all_val_precisions)
    final_avg_val_recall = sum(all_val_recalls) / len(all_val_recalls)
    final_avg_val_f1 = sum(all_val_f1s) / len(all_val_f1s)

    # Calculate standard deviations
    std_train_loss = statistics.stdev(all_train_losses)
    std_train_precision = statistics.stdev(all_train_precisions)
    std_train_recall = statistics.stdev(all_train_recalls)
    std_train_f1 = statistics.stdev(all_train_f1s)

    std_val_loss = statistics.stdev(all_val_losses)
    std_val_precision = statistics.stdev(all_val_precisions)
    std_val_recall = statistics.stdev(all_val_recalls)
    std_val_f1 = statistics.stdev(all_val_f1s)

    print("Training ended...\n")
    print(f"Train Average Loss: {final_avg_train_loss:.4f}, Std: {std_train_loss:.4f}")
    print(f"Train Average Pixel-level Precision: {final_avg_train_precision:.4f}, Std: {std_train_precision:.4f}")
    print(f"Train Average Recall: {final_avg_train_recall:.4f}, Std: {std_train_recall:.4f}")
    print(f"Train Average F score: {final_avg_train_f1:.4f}, Std: {std_train_f1:.4f}\n")
    print(f"Validation Average Loss: {final_avg_val_loss:.4f}, Std: {std_val_loss:.4f}")
    print(f"Validation Average Pixel-level Precision: {final_avg_val_precision:.4f}, Std: {std_val_precision:.4f}")
    print(f"Validation Average Recall: {final_avg_val_recall:.4f}, Std: {std_val_recall:.4f}")
    print(f"Validation Average F score: {final_avg_val_f1:.4f}, Std: {std_val_f1:.4f}")

    if best_model_wts is not None:
        print(f"\nBest model saved from epoch {best_epoch} with Validation F score: {best_f1:.4f}")

    return model

In [ ]:
for inputs, masks, _, _ in train_loader:
    print("Inputs shape:", inputs.shape)
    print("Gold standard shape:", masks.shape)
    print("Input min and max values:", inputs.min().item(), inputs.max().item())
    print("Gold standard unique values:", np.unique(masks.numpy()))
    break

Inputs shape: torch.Size([2, 1, 512, 512])
Gold standard shape: torch.Size([2, 1, 512, 512])
Input min and max values: 0.0 1.0
Gold standard unique values: [0. 1.]


In [ ]:
num_epochs = 20
patience = 5
threshold = 0.75

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.ones_(m.weight)
        nn.init.zeros_(m.bias)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

p = 0.5
model = default_unet(n_channels=1, n_classes=1, dropout_prob=p).to(device)

model.apply(init_weights)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=patience)

trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=scheduler, num_epochs=num_epochs, patience=patience, threshold=threshold)

Training started:



100%|██████████| 60/60 [00:04<00:00, 14.38it/s]


Epoch 1/20 Train Loss: 0.3894
Train Pixel-level Precision: 0.1997
Train Recall: 0.0686
Train F score: 0.0981

Validation Loss: 0.2993
Validation Pixel-level Precision: 0.0000
Validation Recall: 0.0000
Validation F score: 0.0000



100%|██████████| 60/60 [00:04<00:00, 14.66it/s]


Epoch 2/20 Train Loss: 0.3101
Train Pixel-level Precision: 0.6758
Train Recall: 0.3898
Train F score: 0.4839

Validation Loss: 0.4179
Validation Pixel-level Precision: 0.2179
Validation Recall: 0.9257
Validation F score: 0.3438



...

Epoch 18/20 Train Loss: 0.1118
Train Pixel-level Precision: 0.9678
Train Recall: 0.8324
Train F score: 0.8938

Validation Loss: 0.1230
Validation Pixel-level Precision: 0.5882
Validation Recall: 0.5747
Validation F score: 0.5615



100%|██████████| 60/60 [00:04<00:00, 14.54it/s]


Epoch 19/20 Train Loss: 0.1094
Train Pixel-level Precision: 0.9609
Train Recall: 0.8155
Train F score: 0.8790

Validation Loss: 0.1193
Validation Pixel-level Precision: 0.5760
Validation Recall: 0.6123
Validation F score: 0.5849



100%|██████████| 60/60 [00:04<00:00, 14.76it/s]


Epoch 20/20 Train Loss: 0.1031
Train Pixel-level Precision: 0.9702
Train Recall: 0.8444
Train F score: 0.9016

Validation Loss: 0.1409
Validation Pixel-level Precision: 0.5328
Validation Recall: 0.7913
Validation F score: 0.6163

Training ended...

Train Average Loss: 0.1736, Std: 0.0743
Train Average Pixel-level Precision: 0.8936, Std: 0.1787
Train Average Recall: 0.7467, Std: 0.1898
Train Average F score: 0.8080, Std: 0.1929

Validation Average Loss: 0.2417, Std: 0.1109
Validation Average Pixel-level Precision: 0.4273, Std: 0.1531
Validation Average Recall: 0.7979, Std: 0.2223
Validation Average F score: 0.5165, Std: 0.1523

Best model saved from epoch 14 with Validation F score: 0.6268


In [ ]:
model_save_path = '.../path/...'
torch.save(trained_model.state_dict(), model_save_path)

In [ ]:
# LOAD AND TEST MODE

model_path = '.../path/...'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trained_model = default_unet(n_channels=1, n_classes=1).to(device)

trained_model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
def test_model(model, test_loader, threshold=0.2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    precision_metric = BinaryPrecision().to(device)
    recall_metric = BinaryRecall().to(device)
    f1_metric = BinaryF1Score().to(device)

    all_precisions = []
    all_recalls = []
    all_f1s = []

    good_results = []
    acceptable_results = []
    problematic_results = []

    with torch.no_grad():
        for i, (images, masks, image_names, mask_names) in enumerate(test_loader):
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)

            outputs = torch.sigmoid(outputs)
            preds = outputs > threshold

            for j in range(images.size(0)):
                precision = precision_metric(preds[j], masks[j])
                recall = recall_metric(preds[j], masks[j])
                f1 = f1_metric(preds[j], masks[j])

                print(f"Image: {image_names[j]}, Precision: {precision.item():.4f}, Recall: {recall.item():.4f}, F1 Score: {f1.item():.4f}")

                all_precisions.append(precision.item())
                all_recalls.append(recall.item())
                all_f1s.append(f1.item())

                # Good, acceptable, problematic threshold
                result = (images[j].cpu(), masks[j].cpu(), preds[j].cpu(), image_names[j], mask_names[j], f1.item())
                if f1 > 0.85:
                    good_results.append(result)
                elif 0.75 <= f1 <= 0.85:
                    acceptable_results.append(result)
                else:
                    problematic_results.append(result)

    avg_precision = sum(all_precisions) / len(all_precisions)
    avg_recall = sum(all_recalls) / len(all_recalls)
    avg_f1 = sum(all_f1s) / len(all_f1s)

    std_test_precision = statistics.stdev(all_precisions)
    std_test_recall = statistics.stdev(all_recalls)
    std_test_f1 = statistics.stdev(all_f1s)

    print(f"Test Average Precision: {avg_precision:.4f}, Std: {std_test_precision:.4f}")
    print(f"Test Average Recall: {avg_recall:.4f}, Std: {std_test_recall:.4f}")
    print(f"Test Average F1 Score: {avg_f1:.4f}, Std: {std_test_f1:.4f}")

    # pick best ones
    good_results = sorted(good_results, key=lambda x: x[5], reverse=True)[:2]
    acceptable_results = sorted(acceptable_results, key=lambda x: x[5], reverse=True)[:2]
    problematic_results = sorted(problematic_results, key=lambda x: x[5], reverse=True)[:2]

    def gold_pred_plot(results, title):
        for idx, (image, mask, pred, img_name, mask_name, f1_score) in enumerate(results):
            plt.figure(figsize=(15, 5))
            plt.subplot(1, 3, 1)
            plt.imshow(image.squeeze(0), cmap='gray')
            plt.title(f'Original Image: {img_name}')

            plt.subplot(1, 3, 2)
            plt.imshow(mask.squeeze(0), cmap='gray')
            plt.title(f'Gold Standard: {mask_name}')

            plt.subplot(1, 3, 3)
            plt.imshow(pred.squeeze(0), cmap='gray')
            plt.title('Predicted Segmentation')

            plt.suptitle(f'{title} {idx+1} Segmentation - F1 Score: {f1_score:.4f}')
            plt.show()

    gold_pred_plot(good_results, 'GOOD')
    gold_pred_plot(acceptable_results, 'Acceptable')
    gold_pred_plot(problematic_results, 'Problematic')

test_model(trained_model, test_loader, threshold=threshold)